In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import functools
import itertools
import fit_glm_helpers as fgh

# Import Data

Inputted data should conform to the following conventions:

### Session ID
* Must be a unique string file identifier that will represent the combined version of the signal and behavior files

### Signal File
* Each file should represent a single recording with a constant sampling rate
* Each row in the file should represent a single timestamp within a given recording (must be in chronological order)

Example:
| id_timestamp | identifier_1 | predictor_1 | predictor_2 | response_1 | response_2 |
| --- | --- | --- | --- | --- | --- |
| 0 | 0 | 0 | 0 | 1 | 0.3 |
| 1 | 0 | 0 | 0 | 0 | 1.4 |
| 2 | 0 | 0 | 0 | 1 | 2.3 |
| 3 | 0 | 1 | 0 | 1 | 0.3 |
| 4 | 0 | 0 | 0 | 0 | 1.4 |
| 5 | 0 | 0 | 0 | 1 | 2.3 |
| 6 | 1 | 0 | 0 | 0 | 1.4 |
| 7 | 0 | 0 | 0 | 1 | 2.3 |
| 8 | 0 | 0 | 0 | 0 | 1.4 |
| 9 | 1 | 0 | 0 | 1 | 2.3 |
| 10 | 0 | 0 | 0 | 0 | 1.4 |
| 11 | 0 | 0 | 0 | 1 | 2.3 |
| 12 | 0 | 0 | 0 | 0 | 1.4 |
| 13 | 0 | 0 | 0 | 1 | 2.3 |
| 14 | 0 | 0 | 1 | 0 | 1.4 |
| 15 | 0 | 0 | 0 | 0 | 2.3 |
| 16 | 0 | 0 | 0 | 0 | 1.4 |
| 17 | 0 | 0 | 0 | 0 | 2.3 |
| 18 | 0 | 1 | 0 | 0 | 1.4 |
| 19 | 0 | 0 | 0 | 1 | 2.3 |

### Trial File (Optional)
* Each file should represent a single recording with a constant sampling rate
* Each row must represent a unique trial in chronological order, but does not need to start from zero

Columns (Alignment Columns + Information Columns):
* Alignment Columns: Contains the indices (in the signal table) at which the event in question occurs
* Information Columns: Contains information associated with the given trial

Example:
| id_trial | centerInIndex (Alignment) | centerOutIndex (Alignment) | sideInIndex (Alignment) | sideOutIndex (Alignment) | hasAllData (Information) |
| --- | --- | --- | --- | --- | --- |
| trial_0 | 12 | 13 | 17 | 18 | 0 |
| trial_1 | 20 | 22 | 23 | 25 | 1 |
| trial_0 | 27 | 28 | 31 | 35 | 1 |
| trial_2 | 50 | 54 | 57 | 60 | 1 |

In [3]:
dir_data = Path('/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/data/old-data-version/raw-new/Figure_1_2')
dir_output = Path('/Users/josh/Desktop/example_output_folder')

lst_dict_inputdata = [
    {'session_id': 'WT63_11082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT63_11082021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT63_11082021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT63_11102021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT63_11102021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT63_11102021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT63_11122021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT63_11122021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT63_11122021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT63_11162021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT63_11162021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT63_11162021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT63_11182021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT63_11182021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT63_11182021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT64_11082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT64_11082021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT64_11082021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT64_11102021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT64_11102021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT64_11102021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT64_11122021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT64_11122021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT64_11122021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT64_11162021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT64_11162021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT64_11162021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT64_11182021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT64_11182021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT64_11182021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT65_11082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT65_11082021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT65_11082021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT65_11102021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT65_11102021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT65_11102021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT65_11122021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT65_11122021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT65_11122021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT65_11162021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT65_11162021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT65_11162021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT65_11182021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT65_11182021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT65_11182021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT66_12132021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT66_12132021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT66_12132021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT66_12152021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT66_12152021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT66_12152021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT66_12172021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT66_12172021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT66_12172021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT66_12192021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT66_12192021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT66_12192021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT67_12132021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT67_12132021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT67_12132021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT67_12152021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT67_12152021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT67_12152021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT67_12172021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT67_12172021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT67_12172021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT67_12192021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT67_12192021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT67_12192021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT68_12132021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT68_12132021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT68_12132021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT68_12152021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT68_12152021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT68_12152021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT68_12172021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT68_12172021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT68_12172021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT68_12192021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT68_12192021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT68_12192021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT69_12132021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT69_12132021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT69_12132021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT69_12152021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT69_12152021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT69_12152021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT69_12172021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT69_12172021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT69_12172021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT69_12192021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT69_12192021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT69_12192021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA'},
    'columnRenames_trial': None},

    {'session_id': 'WT58_10042021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT58_10042021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT58_10042021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT58_10062021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT58_10062021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT58_10062021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT58_10082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT58_10082021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT58_10082021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT58_10112021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT58_10112021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT58_10112021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT58_10132021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT58_10132021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT58_10132021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT58_10152021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT58_10152021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT58_10152021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT60_10042021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT60_10042021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT60_10042021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT60_10062021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT60_10062021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT60_10062021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT60_10082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT60_10082021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT60_10082021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT60_10112021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT60_10112021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT60_10112021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT60_10132021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT60_10132021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT60_10132021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT60_10152021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT60_10152021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT60_10152021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT61_10042021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT61_10042021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT61_10042021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT61_10062021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT61_10062021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT61_10062021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT61_10082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT61_10082021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT61_10082021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT61_10112021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT61_10112021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT61_10112021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT61_10132021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT61_10132021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT61_10132021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT61_10152021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT61_10152021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT61_10152021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT53_09012021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT53_09012021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT53_09012021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT53_09032021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT53_09032021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT53_09032021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT53_09062021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT53_09062021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT53_09062021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT54_09012021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT54_09012021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT54_09012021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT54_09032021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT54_09032021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT54_09032021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT54_09062021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT54_09062021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT54_09062021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT55_09012021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT55_09012021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT55_09012021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT55_09032021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT55_09032021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT55_09032021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT55_09062021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT55_09062021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT55_09062021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT56_09012021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT56_09012021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT56_09012021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT56_09032021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT56_09032021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT56_09032021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT56_09062021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT56_09062021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT56_09062021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT57_10042021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT57_10042021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT57_10042021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT57_10062021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT57_10062021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT57_10062021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT57_10082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT57_10082021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT57_10082021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT57_10112021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT57_10112021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT57_10112021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT57_10132021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT57_10132021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT57_10132021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT57_10152021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT57_10152021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT57_10152021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT59_10042021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT59_10042021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT59_10042021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT59_10062021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT59_10062021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT59_10062021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT59_10082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT59_10082021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT59_10082021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT59_10112021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT59_10112021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT59_10112021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT59_10132021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT59_10132021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT59_10132021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None},

    {'session_id': 'WT59_10152021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT59_10152021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT59_10152021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch5': 'gACH'},
    'columnRenames_trial': None}
]

dir_output.mkdir(parents=True, exist_ok=True)

In [6]:
columnName_alignment_trial_start = 'photometryCenterInIndex'
columnName_alignment_trial_end = 'photometrySideOutIndex'

# Note: Alignment values of 0 for Matlab-indexed trial tables will be treated as "no-data" values
# and and -1 for Python-indexed trial tables. Matlab-indexed trial tables should only have values
# >= 0 in and >= -1 in Python.
lst_columnNames_trialTable_alignment = [
    'photometryCenterInIndex',
    'photometryCenterOutIndex',
    'photometrySideInIndex',
    'photometrySideOutIndex',
]

lst_columnNames_trialTable_information = [
    'nTrial_raw', 'hasAllPhotometryData',
    'wasRewarded', 'word',
]

In [7]:
lst_columnNamePairs_trial_alignInfo = list(itertools.product(
    lst_columnNames_trialTable_alignment,
    lst_columnNames_trialTable_information,
))

lst_columnNamePairs_trial_alignInfo = (
    [(columnName_alignment, columnName_alignment) for columnName_alignment in lst_columnNames_trialTable_alignment] +
    lst_columnNamePairs_trial_alignInfo
)

In [8]:
bool_drop_zeroAlignments = True
lst_df_signal_all = []

for dict_inputdata in lst_dict_inputdata:
    columnName_trialTable_trialId = dict_inputdata.get('columnName_trialTable_trialId', None)

    # Load data
    df_trial = pd.read_csv(dict_inputdata['filepath_trial'])
    df_signal = pd.read_csv(dict_inputdata['filepath_signal'])
    df_signal = df_signal.rename(dict_inputdata['columnRenames_signal'], axis=1)

    fgh.check_monotonically_increasing_df(df_trial[lst_columnNames_trialTable_alignment], bool_ignoreRowsWithZeros=True, bool_ignoreRowsWithNans=True, equal_allowed=False) # Check that alignment values are monotonically increasing

    df_trial = df_trial[df_trial[lst_columnNames_trialTable_alignment].values.min(axis=1) > 0].copy() if bool_drop_zeroAlignments else df_trial.copy() # Drop trials with no data
    columnName_trialTable_trialId, df_trial = ('nTrial_raw', fgh.add_column_nTrial_raw(df_trial, 'nTrial_raw')) if columnName_trialTable_trialId is None else (columnName_trialTable_trialId, df_trial.copy()) # Add trial ID column if not already present
    
    if not dict_inputdata['bool_trialTable_matlab_indexed']:
        df_trial = df_trial[lst_columnNames_trialTable_alignment] + 1 # Convert to Matlab-indexed trial table
    assert df_trial[lst_columnNames_trialTable_alignment].values.min() >= 0, "All alignment values must be >= 0 for Matlab-indexed trial tables."

    df_signal = pd.concat([pd.DataFrame(pd.Series(0, index=df_signal.columns)).T, df_signal], axis=0).reset_index(drop=True) # Add a row of zeros to the top of the signal dataframe to absorb "no-data" values
    df_alignmentInfoPairs = fgh.alignmentInfoTrialPairs_to_signalColumns(df_trial, lst_columnNamePairs_trial_alignInfo, df_signal.index)

    assert len(set(df_alignmentInfoPairs.columns).intersection(set(df_signal.columns))) == 0, "Alignment info dataframe must not have any columns that appear in the signal dataframe."

    df_signal = pd.concat([df_signal, df_alignmentInfoPairs], axis=1)

    assert len(set([columnName_trialTable_trialId, 'nTrial', 'nEndTrial']).intersection(set(df_signal.columns))) == 0, f"Signal dataframe must not have any columns named 'nTrial', 'nEndTrial', or User Specified: '{columnName_trialTable_trialId}' name."
    df_signal[columnName_trialTable_trialId] = df_signal[f'{columnName_alignment_trial_start}={columnName_trialTable_trialId}'].combine_first(df_signal[f'{columnName_alignment_trial_end}={columnName_trialTable_trialId}'])
    df_signal.insert(0, 'session_id', dict_inputdata['session_id'])
    df_signal.insert(1, 'nTrial', df_signal[columnName_trialTable_trialId].ffill())
    df_signal.insert(2, 'nEndTrial', df_signal[columnName_trialTable_trialId].bfill())
    fgh.check_monotonically_increasing_df(df_signal[['nTrial']], bool_ignoreRowsWithZeros=False, bool_ignoreRowsWithNans=True, equal_allowed=True)
    fgh.check_monotonically_increasing_df(df_signal[['nEndTrial']], bool_ignoreRowsWithZeros=False, bool_ignoreRowsWithNans=True, equal_allowed=True)

    df_signal = df_signal.drop(columnName_trialTable_trialId, axis=1).copy()
    df_signal = df_signal.drop(0, axis=0).copy()
    df_signal.insert(3, 'timestamp', np.arange(df_signal.shape[0])) # / dict_inputdata['sampling_rate']
    lst_df_signal_all.append(df_signal)

df_signal_all = pd.concat(lst_df_signal_all, axis=0).reset_index(drop=True)
df_signal_all.to_csv(dir_output / 'df_signal_all.csv', index=False)

Error reading /Users/josh/Documents/Harvard/GLM/sabatinilab-glm/data/old-data-version/raw-new/Figure_1_2/GLM_SIGNALS_WT63_11082021.txt or /Users/josh/Documents/Harvard/GLM/sabatinilab-glm/data/old-data-version/raw-new/Figure_1_2/GLM_TABLE_WT63_11082021.txt
Successfully read signal and trial table:  GLM_SIGNALS_WT63_11102021.txt GLM_TABLE_WT63_11102021.txt
Successfully read signal and trial table:  GLM_SIGNALS_WT63_11122021.txt GLM_TABLE_WT63_11122021.txt
Successfully read signal and trial table:  GLM_SIGNALS_WT63_11162021.txt GLM_TABLE_WT63_11162021.txt
Successfully read signal and trial table:  GLM_SIGNALS_WT63_11182021.txt GLM_TABLE_WT63_11182021.txt
Successfully read signal and trial table:  GLM_SIGNALS_WT64_11082021.txt GLM_TABLE_WT64_11082021.txt
Successfully read signal and trial table:  GLM_SIGNALS_WT64_11102021.txt GLM_TABLE_WT64_11102021.txt
Successfully read signal and trial table:  GLM_SIGNALS_WT64_11122021.txt GLM_TABLE_WT64_11122021.txt
Successfully read signal and trial t